In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import pandas_access as mdb
import requests
import json
import pickle

################################
# Taxi data (May 2009)
# https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page
################################

# Read in taxi data for May 2009
tlc = pd.read_csv('data/yellow_tripdata_2009-05.csv', 
                 usecols=['Trip_Pickup_DateTime', 'Passenger_Count', 'Start_Lon', 'Start_Lat',
                          'End_Lon', 'End_Lat'])

# Keep only weekdays
tlc.Trip_Pickup_DateTime = pd.to_datetime(tlc.Trip_Pickup_DateTime)
tlc = tlc.loc[tlc.Trip_Pickup_DateTime.dt.dayofweek <= 4, ]

In [2]:
# Read in census tracts
census_tracts = gpd.read_file('data/2000 Census Tracts/geo_export_21ca0b77-873c-4bfe-89c0-462b6aad1ea4.shp')

# Merge in census tract for starting info
start_geometry = [Point(xy) for xy in zip(tlc.Start_Lon, tlc.Start_Lat)]
gdf = gpd.GeoDataFrame(tlc,  geometry=start_geometry)
merged_file = gpd.sjoin(gdf, census_tracts, how='left', op='within')
tlc = pd.DataFrame(merged_file)
# Add county codes
tlc.loc[tlc.boro_name == "Manhattan", "ct2000"] = "061" + tlc.loc[tlc.boro_name == "Manhattan", "ct2000"]
tlc.loc[tlc.boro_name == "Queens", "ct2000"] = "081" + tlc.loc[tlc.boro_name == "Queens", "ct2000"]
tlc.loc[tlc.boro_name == "Brooklyn", "ct2000"] = "047" + tlc.loc[tlc.boro_name == "Brooklyn", "ct2000"]
tlc.loc[tlc.boro_name == "Bronx", "ct2000"] = "005" + tlc.loc[tlc.boro_name == "Bronx", "ct2000"]
tlc.loc[tlc.boro_name == "Staten Island", "ct2000"] = "085" + tlc.loc[tlc.boro_name == "Staten Island", "ct2000"]
tlc = tlc[['Trip_Pickup_DateTime', 'Passenger_Count', 'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat',  'ct2000']]
tlc.columns = ['Trip_Pickup_DateTime', 'Passenger_Count', 'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat', 'start_ct2000']

# Merge in census tract for destination info
end_geometry = [Point(xy) for xy in zip(tlc.End_Lon, tlc.End_Lat)]
gdf = gpd.GeoDataFrame(tlc,  geometry=end_geometry)
merged_file = gpd.sjoin(gdf, census_tracts, how='left', op='within')
tlc = pd.DataFrame(merged_file)
# Add county codes
tlc.loc[tlc.boro_name == "Manhattan", "ct2000"] = "061" + tlc.loc[tlc.boro_name == "Manhattan", "ct2000"]
tlc.loc[tlc.boro_name == "Queens", "ct2000"] = "081" + tlc.loc[tlc.boro_name == "Queens", "ct2000"]
tlc.loc[tlc.boro_name == "Brooklyn", "ct2000"] = "047" + tlc.loc[tlc.boro_name == "Brooklyn", "ct2000"]
tlc.loc[tlc.boro_name == "Bronx", "ct2000"] = "005" + tlc.loc[tlc.boro_name == "Bronx", "ct2000"]
tlc.loc[tlc.boro_name == "Staten Island", "ct2000"] = "085" + tlc.loc[tlc.boro_name == "Staten Island", "ct2000"]
tlc = tlc[['Trip_Pickup_DateTime', 'Passenger_Count', 'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat','start_ct2000', 'ct2000']]
tlc.columns = ['Trip_Pickup_DateTime', 'Passenger_Count', 'Start_Lon', 'Start_Lat', 'End_Lon', 'End_Lat', 'start_ct2000', 'end_ct2000']




/Users/brinaseidel/anaconda3/lib/python3.6/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)
/Users/brinaseidel/anaconda3/lib/python3.6/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != epsg:4326)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [19]:
# Get average hourly passenger counts by start-end tuple
tlc['hour'] = tlc.Trip_Pickup_DateTime.dt.hour
tlc = tlc.groupby(["start_ct2000",  "end_ct2000", "hour"])['Passenger_Count'].sum().reset_index()
# Average across all weekdays in the month
tlc.Passenger_Count = tlc.Passenger_Count/21

/Users/brinaseidel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
################################
# MTA Data (May-Nov 2008)
# http://web.mta.info/mta/planning/data-nyc-travel.html#
################################

# Read in the MTA origin-destination survey data
for tbl in mdb.list_tables("data/NYCSurvey_TripFiles_noXYcoord.mdb"):
    print(tbl)
mta = mdb.read_table("data/NYCSurvey_TripFiles_noXYcoord.mdb", "ltrips_short_exp123_wkdy_final_noXYcoord", converters_from_schema=False)

# Use trip-level weights (EXP32_FINAL_WKD) as person counts
mta = mta[['EXP32_FINAL_WKD', 'dtime','trip_sdate', 'O_TRACT', 'D_TRACT']]

# Format census tracts to get six digits
mta.D_TRACT = mta.D_TRACT.apply(str).str.zfill(13)
mta.D_TRACT = mta.D_TRACT.str.slice(2, 11)
mta.O_TRACT = mta.O_TRACT.apply(str).str.zfill(13)
mta.O_TRACT = mta.O_TRACT.str.slice(2, 11)

# Get hour
mta['hour'] = mta.dtime.str.slice(0,2).apply(int)

# Get hourly sums by start-end tuple
mta = mta.groupby(['O_TRACT', 'D_TRACT', 'hour']).EXP32_FINAL_WKD.sum().reset_index()
mta.EXP32_FINAL_WKD = mta.EXP32_FINAL_WKD/5 # these are weekday approximations; we want daily

ltrips_short_exp123_wkdy_final_noXYcoord
ltrips_short_exp13_sat_final_noXYcoord
ltrips_short_exp13_sun_final_noXYcoord


In [21]:
# Merge
mta.columns = ['origin_tract', 'dest_tract', 'hour', 'mta_trips']
tlc.columns = ['origin_tract', 'dest_tract', 'hour', 'tlc_trips']
trips = mta.merge(tlc, on=['origin_tract', 'dest_tract', 'hour'], how="outer")

In [22]:
# Save to pickle
pickle.dump(trips, open( "data/trips_long.p", "wb" ) )
trips

,origin_tract,dest_tract,hour,mta_trips,tlc_trips
0,001000403,081129101,16,190.366778,NaN
1,001001100,061022000,16,17.680145,NaN
2,001001100,081002900,9,158.542912,NaN
3,001001100,081009900,16,61.459843,NaN
4,001005900,047003300,11,119.420210,NaN
...,...,...,...,...,...
1341415,085032300,085022600,19,NaN,0.095238
1341416,085032300,085030302,4,NaN,0.047619
1341417,085032300,085032300,0,NaN,0.190476
1341418,085032300,085032300,1,NaN,0.047619


In [122]:
# Get daily total trips
daily_total = trips.groupby(['origin_tract', 'dest_tract'])[['mta_trips', 'tlc_trips']].sum().reset_index()
daily_total = daily_total.fillna(0)
daily_total['trips'] = daily_total.mta_trips + daily_total.tlc_trips


In [123]:
# P0030001: Total population
params = {"key": "7530e2501288a8dfb28803342f5d1493cf00cb96",
          "state": "36",   # New York
          "county": "061,047,005,085,081",  #New York County (Manhattan), Kings County (Brooklyn), Bronx County (The Bronx), Richmond County (Staten Island), and Queens County (Queens)
          "indicators": "P001001"
         }
url = "https://api.census.gov/data/2000/sf1?get="+params["indicators"]+"&for=tract:*&in=state:"+params["state"]+"&in=county:"+params["county"]+"&key="+params["key"]
response = requests.get(url, data = {'key':'value'})
colnames = response.json()[0]
colnames[0] = "population"
population = response.json()[1:]
population = pd.DataFrame(population, columns=colnames)
population.tract = population.county + population.tract # Add county code
population = population[['population', 'tract']]
population.population = population.population.apply(int)
# Drop areas with no population
population = population.loc[population.population != 0, ]

In [124]:
# Cross join population with itself to make a long version of the origin-destination matrix
population['key']  = 0
od_long = population.merge(population, on='key', how='outer')
od_long = od_long[['tract_x', 'tract_y', 'population_x']]
od_long.columns = ['origin_tract', 'dest_tract', 'population']

# Keep population only for cases where origin=destination (diagonal of OD matrix)
od_long.loc[od_long.origin_tract != od_long.dest_tract, 'population'] = 0

# Subtract total pop that leaves each origin tract during the day
pop_leaving = daily_total.groupby('origin_tract').trips.sum().reset_index()
od_long = od_long.merge(pop_leaving, on='origin_tract', how='left')
od_long.loc[od_long.origin_tract == od_long.dest_tract, 'population'] = od_long.loc[od_long.origin_tract == od_long.dest_tract, 'population'] - od_long.loc[od_long.origin_tract == od_long.dest_tract, 'trips'] 
od_long.drop(['trips'], axis=1, inplace=True)

# 

In [125]:
# Merge in pop moving from each origin to each destination
od_long = od_long.merge(daily_total, on=['origin_tract', 'dest_tract'], how='left')
od_long.loc[od_long.origin_tract != od_long.dest_tract, 'population'] =  od_long.loc[od_long.origin_tract != od_long.dest_tract, 'trips'] 


In [126]:
# Reshape
od = od_long.pivot(index='origin_tract', columns='dest_tract', values='population').reset_index().fillna(0)
# Add one to every cell for smoothing purposes
od[od.columns[1:]] = od[od.columns[1:]]  + 1

In [127]:
# Save origin-destination matrix
pickle.dump(od, open( "data/nyc_od.p", "wb" ) )
od.drop("origin_tract", axis=1).to_csv("data/nyc_od.csv", index=False, header=False )